<a href="https://colab.research.google.com/github/rhqtmfajfl/python-study/blob/master/tensorflow/%EB%B3%B4%EB%84%88%EC%8A%A42%5D%ED%81%B4%EB%9E%98%EC%8A%A4%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%84%A4%ED%8A%B8%EC%9B%8C%ED%81%AC_%EA%B5%AC%EC%A1%B0%EC%A0%95%EC%9D%980702.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사용자 정의 모델링
* `tf.keras.Model`클래스를 상속 받아서 모델을 구현할 수 있다.
  * 생성자`__init__` 함수에선느 사용할 레이어를 정의합니다.
  * `call()` 함수에서는 `__init__` 에서 정의한 레이어를 엮어줍니다.

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):

  ##__init__에서는 사용할 레이어들을 '정의'만 한다.

  def __init__(self):
    super(MyModel, self).__init__() # 강제로 부모 클래스의 생성자를 호출해서 부모클래스의 모든 내용을 사용할 수 있도록 함.
    
    #아주 간단한 FCN 구현하기
    self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')
# 자동 미분 시스템ㄷ이 자동으로 순서대로 순전파를 실행하준다.
  
  #call에서는 __init__에서 정의한 레이어를 엮어준다. 결과를 "반드시" 리턴해야 한다.
  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    out = self.d2(x)

    return out

#학습루프 정의 하기


In [ ]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  '''
    model : 학습 대상 모델
    images : 학습에 사용할 이미지 텐서
    labels : 이미지에 맞는 레이블 텐서
    loss_object : 손실함수
    optimizer : 최적화 함수
    train_loss : 훈련 과정 Loss 평가 지표 설정
    train_accuracy : 훈련 과정 accuracy 평가 지표 설정
  '''

  with tf.GradientTape() as tape:
    prediction = model(images) # 모델이 훈련모드 예측을 하게 된다.
    loss =  loss_object(labels, prediction)

  grads = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, prediction)

  # 천천히 읽어보면 됨


# 테스트 루프 정의하기

In [ ]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuarcy):
  prediction = model(images)

  t_loss = loss_object(labels, prediction)

  test_loss(t_loss)
  test_accuracy(labels, prediction)

# 데이터 세트 로딩 및 전처리

In [ ]:
mnist = tf.keras.datasets.mnist

# MNIST 데이터세트 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 픽셀 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

# 이미지 채널 차원 추가
x_train = x_train[..., tf.newaxis]
x_test  = x_test[..., tf.newaxis]

# Tensorflow Dataset 만들기
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 모델 생성(네트워크 생성)

In [ ]:
model = MyModel()


# 손실함수, 최적화 선택하기

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer   = tf.keras.optimizers.Adam()

# 알고리즘 평가지표 설정(performance metrics)


In [ ]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuarcy")

# 학습 진행하기

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
  
  #훈련
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
  
  # 테스트
  for test_images, test_labels in test_ds:
    test_step(model, images, labels, loss_object, test_loss, test_accuracy)

  template = "Epoch {}, Train Loss : {:.3f}, Train Accuracy : {:.3f} 😀 Test Loss : {:.3f}, Test Accuracy : {:.3f}"
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))


Epoch 1, Train Loss : 0.254, Train Accuracy : 92.660 😀 Test Loss : 0.012, Test Accuracy : 100.000
Epoch 2, Train Loss : 0.183, Train Accuracy : 94.651 😀 Test Loss : 0.066, Test Accuracy : 98.438
Epoch 3, Train Loss : 0.148, Train Accuracy : 95.680 😀 Test Loss : 0.229, Test Accuracy : 96.875
Epoch 4, Train Loss : 0.125, Train Accuracy : 96.327 😀 Test Loss : 0.172, Test Accuracy : 97.656
Epoch 5, Train Loss : 0.109, Train Accuracy : 96.799 😀 Test Loss : 0.144, Test Accuracy : 98.125
Epoch 6, Train Loss : 0.096, Train Accuracy : 97.161 😀 Test Loss : 0.120, Test Accuracy : 98.438
Epoch 7, Train Loss : 0.087, Train Accuracy : 97.450 😀 Test Loss : 0.103, Test Accuracy : 98.661
Epoch 8, Train Loss : 0.079, Train Accuracy : 97.692 😀 Test Loss : 0.093, Test Accuracy : 98.828
Epoch 9, Train Loss : 0.072, Train Accuracy : 97.883 😀 Test Loss : 0.083, Test Accuracy : 98.958
Epoch 10, Train Loss : 0.066, Train Accuracy : 98.052 😀 Test Loss : 0.075, Test Accuracy : 99.062
